In [ ]:
from IPython.display import display
from owslib.wms import WebMapService
import math
import cv2
import numpy as np
import urllib.request

In [59]:
import geopandas as gpd
import fiona
import os
gdb_path = os.path.join('..', 'data', 'Wildtierpassagen.gdb') if not os.path.exists('data/Wildtierpassagen.gdb') else 'data/Wildtierpassagen.gdb'
print('Using geodatabase path:', gdb_path)
# List available layers
layers = fiona.listlayers(gdb_path)
print('Layers found:', layers)

selected_layers = ["N2023_Version_Wildtierpassagen"]

# Read point-type layers into GeoDataFrames
point_gdfs = {}
for lyr in selected_layers:
    try:
        gdf = gpd.read_file(gdb_path, layer=lyr)
        if gdf.geom_type.isin(['Point','MultiPoint']).any():
            point_gdfs[lyr] = gdf
            print(f'Loaded layer {lyr} with {len(gdf)} features; geometry types: ', gdf.geom_type.unique())
        else:
            print(f'Skipping layer {lyr}: no point geometries')
    except Exception as e:
        print(f'Could not read layer {lyr}:', e)
# If no point layers found, stop
if not point_gdfs:
    raise RuntimeError('No point layers found or geodatabase could not be read.')
# Prepare a folium map centered on the combined bounds
from shapely.geometry import Point
import folium
# Merge bounds
minx=miny=maxx=maxy=None
for gdf in point_gdfs.values():
    # ensure has CRS and convert to EPSG:4326
    if gdf.crs is None:
        print('Layer has no CRS; assuming EPSG:4326')
        gdf = gdf.set_crs(epsg=4326)
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)
    if minx is None:
        minx, miny, maxx, maxy = bounds
    else:
        minx = min(minx, bounds[0])
        miny = min(miny, bounds[1])
        maxx = max(maxx, bounds[2])
        maxy = max(maxy, bounds[3])
# Center map
center = [(miny+maxy)/2, (minx+maxx)/2]
m = folium.Map(location=center, zoom_start=12)
# Add each layer as a CircleMarker layer group
for name, gdf in point_gdfs.items():
    # ensure in EPSG:4326 for folium
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    fg = folium.FeatureGroup(name=name)
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue
        # handle MultiPoint by iterating
        if geom.geom_type == 'MultiPoint':
            for pt in geom.geoms:
                folium.CircleMarker(location=[pt.y, pt.x], radius=4, color='red', fill=True).add_to(fg)
        elif geom.geom_type == 'Point':
            folium.CircleMarker(location=[geom.y, geom.x], radius=4, color='blue', fill=True).add_to(fg)
    fg.add_to(m)
folium.LayerControl().add_to(m)
# Display map in notebook
display(m)

Using geodatabase path: ..\data\Wildtierpassagen.gdb
Layers found: ['N2023_Version_Wildtierpassagen', 'SanierungAstra_Catalogue', 'Bauwerkstyp_Catalogue']
Loaded layer N2023_Version_Wildtierpassagen with 63 features; geometry types:  ['Point']


In [ ]:
wms_url = "https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities"
wms = WebMapService(wms_url)

In [35]:
layer_name = "ch.swisstopo.swissimage"
wms[layer_name].title

'SWISSIMAGE Hintergrund'

In [77]:
# For every point in the selected layer(s), request a 200x200 m WMS crop centered on the point
import os
from shapely.geometry import Point
# Output directory for crops
out_dir = os.path.join("..", 'data', 'wms_crops')
os.makedirs(out_dir, exist_ok=True)
# Size in pixels for each crop (200 m x 200 m)
size_pixels = (1024, 1024)  # width, height in px
img_format = 'image/png'
half_size_m = 100  # half of 200 m box

if 'layer_name' not in globals():
    raise RuntimeError('Please define the WMS layer name in variable `layer_name` before running this cell')

# Iterate through the loaded point GeoDataFrames
for layer, gdf in point_gdfs.items():
    print(f'Processing layer {layer} with {len(gdf)} features')
    # Work on a copy and ensure we have a CRS
    g = gdf.copy()
    if g.crs is None:
        print(' - layer has no CRS, assuming EPSG:4326')
        g = g.set_crs(epsg=4326)
    # Reproject to the projected CRS used by the WMS (EPSG:2056) so bbox units are meters
    try:
        g2056 = g.to_crs(epsg=2056)
    except Exception as e:
        print(' - CRS transform to EPSG:2056 failed for layer', layer, e)
        continue
    for idx, row in g2056.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue
        pts = []
        if geom.geom_type == 'Point':
            pts = [geom]
        else:
            # skip non-point geometries
            continue

        for p_i, pt in enumerate(pts):
            x, y = pt.x, pt.y
            bbox = (x - half_size_m, y - half_size_m, x + half_size_m, y + half_size_m)
            try:
                img = wms.getmap(
                    layers=[layer_name],
                    srs='EPSG:2056',
                    bbox=bbox,
                    size=size_pixels,
                    format=img_format,
                    transparent=True,
                )
                # filename: <layername>_<featureindex>[_ptindex].png
                safe_layer = layer.replace(' ', '_')
                fname = f'{safe_layer}_{idx}' + (f'_pt{p_i}' if p_i else '') + '.png'
                outpath = os.path.join(out_dir, fname)
                with open(outpath, 'wb') as f:
                    f.write(img.read())
                # print(f'Saved {outpath} (bbox={bbox})')
            except Exception as e:
                print(f'Failed to fetch image for {layer} index {idx} pt{p_i}:', e)

Processing layer N2023_Version_Wildtierpassagen with 63 features


In [ ]:
# url list contains the whole swissimage 10cm url list

with open(r"C:\code\cassda-zertifikatsarbeit\data\swissimage_10cm_urllist.txt", "r") as f:
    url_list = [line.strip() for line in f if line.strip()]

In [ ]:
for idx, row in g2056.iterrows():
        
        if idx > 5:
            break
        
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue
        pts = []
        if geom.geom_type == 'Point':
            pts = [geom]
        else:
            # skip non-point geometries
            continue

        for p_i, pt in enumerate(pts):
            x, y = math.floor(pt.x / 10000)*10, math.floor(pt.y / 10000)*10
            try:
                img_url = [url for url in url_list if f"{x}-{y}" in url][0]
                img_data = urllib.request.urlopen(img_url).read()
                cv2_img = cv2.imdecode(np.frombuffer(img_data, np.uint8), cv2.IMREAD_UNCHANGED)
                cv2.imwrite(os.path.join(out_dir, f'swissimage_10cm_{idx}' + (f'_pt{p_i}' if p_i else '') + '.png'), cv2_img)
                    
            except Exception as e:
                print(f'Failed to fetch image for {layer} index {idx} pt{p_i}:', e)

'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2023_2540-1190/swissimage-dop10_2023_2540-1190_0.1_2056.tif'

In [86]:
f"{x}-{y}"

'2540-1190'